In [1]:
# Core libraries
import pandas as pd
import numpy as np
import os
import warnings

# Statistical analysis
from scipy import stats
from scipy.stats import chi2_contingency, ttest_ind
import statsmodels.api as sm

# Machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import altair as alt

# Environment and database
from dotenv import load_dotenv
from supabase import create_client

# Settings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)


In [2]:
# load the file
load_dotenv(dotenv_path="D:/Work/Projects/Notebooks/talent-match-intel/streamlit_app/key.env")

# read without printing secrets
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)

print("✅ Connected to Supabase")

✅ Connected to Supabase


In [3]:
def load_table(table_name, chunk_size=1000):
    all_data = []
    offset = 0
    
    print(f"🔄 Starting unlimited load for: {table_name}")
    
    while True:
        try:
            # Fetch data using .range() starting from the current offset
            response = (
                supabase.table(table_name)
                .select("*")
                .range(offset, offset + chunk_size - 1)
                .execute()
            )
            
            data = response.data
            
            # If the response is empty, we've loaded all records
            if not data:
                break
                
            # Add the fetched chunk of data to the master list
            all_data.extend(data)
            
            # Increment the offset to prepare for the next chunk
            offset += chunk_size
            
            print(f"   -> Fetched {len(data)} records. Total: {len(all_data)}")

        except Exception as e:
            print(f"❌ Error loading {table_name} at offset {offset}: {e}")
            break # Exit the loop on error

    df = pd.DataFrame(all_data)
    print(f"✅ Loaded {table_name}: {len(df)} records in total.")
    return df

In [4]:
# Load all tables
df_employees = load_table('employees')
df_performance = load_table('performance_yearly')
df_competencies = load_table('competencies_yearly')
df_psych = load_table('profiles_psych')
df_papi = load_table('papi_scores')
df_strengths = load_table('strengths')
df_pillars = load_table('dim_competency_pillars')
df_grades = load_table('dim_grades')
df_positions = load_table('dim_positions')
df_education = load_table('dim_education')
df_departments = load_table('dim_departments')
df_areas = load_table('dim_areas')
df_divisions = load_table('dim_divisions')
df_directorates = load_table('dim_directorates')
df_majors = load_table('dim_majors')


🔄 Starting unlimited load for: employees
   -> Fetched 1000 records. Total: 1000
   -> Fetched 1000 records. Total: 2000
   -> Fetched 10 records. Total: 2010
✅ Loaded employees: 2010 records in total.
🔄 Starting unlimited load for: performance_yearly
   -> Fetched 1000 records. Total: 1000
   -> Fetched 1000 records. Total: 2000
   -> Fetched 1000 records. Total: 3000
   -> Fetched 1000 records. Total: 4000
   -> Fetched 1000 records. Total: 5000
   -> Fetched 1000 records. Total: 6000
   -> Fetched 1000 records. Total: 7000
   -> Fetched 1000 records. Total: 8000
   -> Fetched 1000 records. Total: 9000
   -> Fetched 1000 records. Total: 10000
   -> Fetched 50 records. Total: 10050
✅ Loaded performance_yearly: 10050 records in total.
🔄 Starting unlimited load for: competencies_yearly
   -> Fetched 1000 records. Total: 1000
   -> Fetched 1000 records. Total: 2000
   -> Fetched 1000 records. Total: 3000
   -> Fetched 1000 records. Total: 4000
   -> Fetched 1000 records. Total: 5000
   -

# Analysis

## Competency Pillars Analysis

In [5]:
# 1. Merge Competency Scores with Competency Pillars/Labels
df_comp_analysis = df_competencies.merge(
    df_pillars[['pillar_code', 'pillar_label']],
    on='pillar_code',
    how='left'
)

# 2. Merge Competency Data with Final Employee Rating
# Assuming 'employee_id' is the join key
df_comp_analysis = df_comp_analysis.merge(
    df_performance[['employee_id', 'rating']],
    on='employee_id',
    how='inner'
)

# 3. Aggregate Competency Scores
# For a holistic view, group the competency scores by Pillar and Rating for T-test preparation.
df_comp_analysis_ready = df_comp_analysis.dropna(subset=['pillar_label', 'rating', 'score'])

In [6]:
# 1. CRITICAL FIX: Convert the 'score' column to numeric, forcing errors to NaN.
df_comp_analysis_ready['score'] = pd.to_numeric(df_comp_analysis_ready['score'], errors='coerce')


# Define Cohen's d Function (for effect size)
def calculate_cohens_d(group1, group2):
    """Calculates Cohen's d for two independent groups."""
    # ... (function body remains the same) ...
    n1, n2 = len(group1), len(group2)
    s1, s2 = group1.std(), group2.std()
    
    if n1 + n2 - 2 <= 0: return np.nan
    sp = np.sqrt(((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2))
    if sp == 0: return np.nan
        
    d = (group1.mean() - group2.mean()) / sp
    return d

competency_test_results = []
competency_list = df_comp_analysis_ready['pillar_label'].unique()
MIN_SAMPLE_SIZE = 10 

for pillar in competency_list:
    # Separate the scores for High (5) and Low (1/2) Performers for the current pillar
    # NOTE: .dropna() is still essential here to remove the NaN values created by pd.to_numeric
    high_scores = df_comp_analysis_ready[
        (df_comp_analysis_ready['pillar_label'] == pillar) & 
        (df_comp_analysis_ready['rating'] == 5.0)
    ]['score'].dropna()
    
    low_scores = df_comp_analysis_ready[
        (df_comp_analysis_ready['pillar_label'] == pillar) & 
        (df_comp_analysis_ready['rating'].isin([1.0, 2.0]))
    ]['score'].dropna()
    
    # Proceed only if both groups have sufficient data
    if len(high_scores) >= MIN_SAMPLE_SIZE and len(low_scores) >= MIN_SAMPLE_SIZE:
        
        # Welch's T-test (safer when variances are unequal)
        # This line will now work because high_scores and low_scores are numeric Series
        t_stat, p_value = stats.ttest_ind(high_scores, low_scores, equal_var=False) 
        
        # Calculate Effect Size
        cohens_d = calculate_cohens_d(high_scores, low_scores)
        
        competency_test_results.append({
            'Competency_Pillar': pillar,
            'Mean_R5': high_scores.mean(),
            'Mean_R12': low_scores.mean(),
            'Cohens_d': cohens_d,
            'P_Value': p_value,
            'Significant': p_value < 0.05 
        })

df_comp_results = pd.DataFrame(competency_test_results)

In [7]:
# Filter for statistically significant results AND sort by Cohen's d
df_final_ranking = (
    df_comp_results
    .sort_values('Cohens_d', ascending=False)
)

print("--- 🏆 Top Competency Differentiators (High vs. Low Performers) ---")
print("Ranked by Cohen's d (Largest Performance Gap)")
df_final_ranking['Significant'] = df_final_ranking['Significant'].apply(lambda x: '✅' if x else '❌')
print(df_final_ranking.round(3).head(10).to_string())

--- 🏆 Top Competency Differentiators (High vs. Low Performers) ---
Ranked by Cohen's d (Largest Performance Gap)
              Competency_Pillar  Mean_R5  Mean_R12  Cohens_d  P_Value Significant
6      Value Creation for Users    3.588     2.990     0.160      0.0           ✅
2  Insight & Decision Sharpness    3.536     2.968     0.156      0.0           ✅
1   Curiosity & Experimentation    3.467     2.952     0.152      0.0           ✅
5    Social Empathy & Awareness    3.672     3.027     0.150      0.0           ✅
0     Growth Drive & Resilience    3.467     2.963     0.150      0.0           ✅
3   Quality Delivery Discipline    3.768     3.051     0.141      0.0           ✅
8    Forward Thinking & Clarity    3.592     3.016     0.136      0.0           ✅
9     Commercial Savvy & Impact    3.495     3.009     0.126      0.0           ✅
7       Lead, Inspire & Empower    3.672     3.085     0.119      0.0           ✅
4    Synergy & Team Orientation    3.613     3.074     0.113      0

In [8]:
# Sort by Cohen's d descending for visualization consistency
df_comp_results = df_comp_results.sort_values(by='Cohens_d', ascending=False)

# Get the order of pillars based on Cohen's d for consistent ordering across both charts
pillar_order = df_comp_results['Competency_Pillar'].tolist()

# --- 2. Visualization 1: Effect Size (Cohen's d) Chart (Primary View) ---

chart_cohens_d = alt.Chart(df_comp_results).mark_bar(color='#1f77b4').encode(
    # X-axis shows the magnitude of the gap
    x=alt.X('Cohens_d', title="Performance Gap (Cohen's d)", scale=alt.Scale(domain=[0.1, 0.17])),
    # Y-axis lists the competencies, sorted by the gap size
    y=alt.Y('Competency_Pillar', title="Competency Pillar", sort='-x'),
    # Tooltip provides detail on hover
    tooltip=['Competency_Pillar', alt.Tooltip('Cohens_d', format=".3f"), 'P_Value']
).properties(
    title="🏆 Top 10 Competency Differentiators (Ranked by Effect Size)"
).interactive() # Allows interactive zooming/panning

# --- 3. Visualization 2: Mean Score Comparison Chart (Contextual View) ---

# Melt the data to compare Mean_R5 and Mean_R12 side-by-side
df_means = df_comp_results.melt(
    id_vars=['Competency_Pillar', 'Cohens_d'],
    value_vars=['Mean_R5', 'Mean_R12'],
    var_name='Performer_Group',
    value_name='Mean_Score'
).replace({'Mean_R5': 'High Performers (R5)', 'Mean_R12': 'Low Performers (R1/R2)'})


chart_means = alt.Chart(df_means).mark_bar(opacity=0.8).encode(
    # X-axis shows the actual mean score
    x=alt.X('Mean_Score', title="Mean Competency Score", axis=alt.Axis(format=".2f"), scale=alt.Scale(domain=[2.5, 4.0])),
    # Y-axis groups the bars, ordered by the Cohen's d ranking
    y=alt.Y('Competency_Pillar', sort=pillar_order, title="Competency Pillar"),
    # Color distinguishes the performance groups
    color=alt.Color('Performer_Group', title="Group"),
    # Columns create separate panels for the two groups
    column=alt.Column('Performer_Group', header=alt.Header(titleOrient="bottom", labelOrient="bottom")),
    tooltip=['Competency_Pillar', 'Performer_Group', alt.Tooltip('Mean_Score', format=".3f")]
).properties(
    title="Mean Competency Score Comparison"
).interactive()

# --- 4. Display Charts ---
chart_cohens_d | chart_means

alt.HConcatChart(...)

**Insights and key finding**

> We compared the skill scores of our Highest Performers (Rating 5) against our Lowest Performers (Rating 1 or 2) across ten core competencies.

> Finding: The difference between our best and worst employees is not just that the best are slightly better at everything, it’s that they are significantly better at three specific things that drive business results.

> Top 3 Skills to Prioritize for Performance
1. Value Creation for Users (The Biggest Gap). Top performers are uniquely skilled at turning effort into visible, tangible impact for clients or internal stakeholders. They don't just complete tasks, they deliver measurable value that matters to the user.
    Actionable Insight: If you hire, promote, or coach based on this skill, you will see the fastest results in increasing overall top performance.

2. Insight & Decision Sharpness (The Strategic Advantage)
    The best employees are significantly better at analyzing complex situations, finding the root cause of a problem, and making correct, fast decisions. They provide clarity where others find confusion.
    Actionable Insight: This is the cognitive engine of your top performers. Focus leadership training on strategic thinking, data analysis, and effective risk assessment.

3. Curiosity & Experimentation (The Engine of Innovation)
    Top performers aren't content with the status quo. They show a stronger drive to learn new things, challenge assumptions, and try new approaches.
    Actionable Insight: Foster a culture where testing new ideas (and failing fast) is rewarded. This is the skill that fuels continuous improvement and innovation within the company.

> Conclusion: Most employees are already competent at being good team players and delivering basic quality. These skills are the entry ticket to the job, but they will not be the skills that determine who gets the top Rating 5. The key is pushing past the basics into strategic impact.

## Psychometric Profiles Analysis

In [ ]:
# --- Re-define Cohen's d Function ---
def calculate_cohens_d(group1, group2):
    """Calculates Cohen's d for two independent groups."""
    n1, n2 = len(group1), len(group2)
    s1, s2 = group1.std(), group2.std()
    
    # Handle the case where groups are too small or have zero variance
    if n1 + n2 - 2 <= 0: return np.nan
    
    sp = np.sqrt(((n1 - 1) * s1**2 + (n2 - 1) * s2**2) / (n1 + n2 - 2))
    
    if sp == 0: return np.nan
        
    d = (group1.mean() - group2.mean()) / sp
    return d


# A. Merge PAPI scores with Rating
df_papi_with_rating = df_papi.merge(
    df_performance[['employee_id', 'rating']], on='employee_id', how='left'
)

# B. Merge with Psych scores (creates a temp master table)
psych_cols = ['employee_id', 'pauli', 'faxtor', 'iq', 'gtq', 'tiki', 'mbti', 'disc'] 
df_master_psych_analysis = df_papi_with_rating.merge(
    df_psych[psych_cols], on='employee_id', how='left'
)

# C. Pivot PAPI scores (from long to wide format)
df_papi_pivoted = df_master_psych_analysis.groupby(['employee_id', 'scale_code'])['score'].mean().unstack()

# D. Prepare psych/rating data
df_psych_with_rating = df_master_psych_analysis[
    ['employee_id', 'rating', 'pauli', 'faxtor', 'iq', 'gtq', 'tiki']
].drop_duplicates(subset=['employee_id', 'rating'])

# E. Final Merge to create df_analysis
df_analysis = df_papi_pivoted.merge(
    df_psych_with_rating,
    on='employee_id',
    how='inner'
)

PAPI_SCALES = [col for col in df_analysis.columns if col.startswith('Papi_')]
PSYCH_METRICS = ['pauli', 'iq', 'gtq', 'tiki'] # Numeric psych metrics
ALL_METRICS_TO_TEST = PAPI_SCALES + PSYCH_METRICS

test_results_r5_r12 = []
HIGH_PERFORMER_RATING = 5.0
LOW_PERFORMER_RATINGS = [1.0, 2.0]  # Expanded to include ratings 1, 2, 3, and 4
MIN_SAMPLE_SIZE = 5

df_low_performers = df_analysis[df_analysis['rating'].isin(LOW_PERFORMER_RATINGS)]

for metric in ALL_METRICS_TO_TEST:
    if metric not in df_analysis.columns:
        continue
        
    high = df_analysis[df_analysis['rating'] == HIGH_PERFORMER_RATING][metric].dropna()
    low = df_low_performers[metric].dropna() 
    
    if len(high) >= MIN_SAMPLE_SIZE and len(low) >= MIN_SAMPLE_SIZE:
        
        t_stat, p_value = stats.ttest_ind(high, low, equal_var=False) 
        cohens_d = calculate_cohens_d(high, low)
        
        test_results_r5_r12.append({
            'Metric': metric,
            'Mean_High': high.mean(),
            'Mean_Low': low.mean(),
            'Difference': high.mean() - low.mean(),
            'P_Value': p_value,
            'Cohens_d': cohens_d,
            'Significant': p_value < 0.05
        })

df_r5_r12_results = pd.DataFrame(test_results_r5_r12)

# Sort and display the results
df_r5_r12_top_effects = (
    df_r5_r12_results
    .iloc[df_r5_r12_results['Cohens_d'].abs().argsort()[::-1]]
    .head(10)
)

print("\n--- TOP 10 TALENT DIFFERENTIATORS (RATING 5 vs. RATING 1 & 2) ---")
df_r5_r12_top_effects['Significant'] = df_r5_r12_top_effects['Significant'].apply(lambda x: '✅' if x else '❌')
print(df_r5_r12_top_effects[['Metric', 'Mean_High', 'Mean_Low', 'Cohens_d', 'P_Value', 'Significant']].round(3).to_string())


--- TOP 10 TALENT DIFFERENTIATORS (RATING 5 vs. RATING 1 & 2) ---
    Metric  Mean_High  Mean_Low  Cohens_d  P_Value Significant
22     gtq     27.954    27.278     0.108    0.027           ✅
11  Papi_O      4.997     4.846     0.059    0.201           ❌
15  Papi_T      4.861     5.014    -0.059    0.199           ❌
0   Papi_A      4.962     5.090    -0.050    0.269           ❌
19  Papi_Z      4.912     5.034    -0.048    0.303           ❌
12  Papi_P      5.054     4.944     0.042    0.357           ❌
9   Papi_L      5.076     5.173    -0.037    0.419           ❌
7   Papi_I      4.953     5.043    -0.035    0.442           ❌
14  Papi_S      5.034     4.944     0.034    0.452           ❌
20   pauli     60.929    60.287     0.027    0.533           ❌


In [ ]:
CATEGORICAL_PSYCH_METRICS = ['disc', 'mbti']
HIGH_PERFORMER_RATING = 5.0

# CRITICAL CORRECTION: Focus on the true "Low" performers to maximize contrast.
LOW_PERFORMER_RATINGS = [1.0, 2.0] 

categorical_test_results = []

# Create the two comparative groups
df_high = df_master_psych_analysis[df_master_psych_analysis['rating'] == HIGH_PERFORMER_RATING].copy()
df_low = df_master_psych_analysis[df_master_psych_analysis['rating'].isin(LOW_PERFORMER_RATINGS)].copy()

# Combine only the two contrasting groups
df_combined = pd.concat([df_high, df_low])

# Loop through each categorical metric
for metric in CATEGORICAL_PSYCH_METRICS:
    if metric not in df_combined.columns:
        continue
        
    # Drop NaNs and ensure we have enough data
    df_temp = df_combined[[metric, 'rating']].dropna()
    
    # Create the contingency table
    contingency_table = pd.crosstab(df_temp[metric], df_temp['rating'])
    
    # Ensure there are at least two rows and two columns to run the test
    if contingency_table.shape[0] > 1 and contingency_table.shape[1] > 1:
        
        # --- Chi-Squared Test ---
        chi2, p_value, dof, expected = chi2_contingency(contingency_table)
    
        
        # Calculate the distribution of categories within the High Performer group (Rating 5.0)
        high_dist = contingency_table[HIGH_PERFORMER_RATING].div(
            contingency_table[HIGH_PERFORMER_RATING].sum()
        ) * 100
        
        # Find the category most prevalent among High Performers
        top_category = high_dist.idxmax()
        top_percentage = high_dist.max()
        
        categorical_test_results.append({
            'Metric': metric,
            'Chi2': chi2,
            'P_Value': p_value,
            'Dominant_Category_in_R5': top_category,
            'R5_Category_Pct': top_percentage,
            'Significant': p_value < 0.05
        })

df_categorical_results = pd.DataFrame(categorical_test_results)

print("\n--- CATEGORICAL PSYCHOMETRICS (R5 vs R1/R2) Chi-Squared Test ---")
df_categorical_results['Significant'] = df_categorical_results['Significant'].apply(lambda x: '✅' if x else '❌')

# Sort by P_Value to show the most significant results first
print(df_categorical_results.sort_values('P_Value').round(4).to_string())


--- CATEGORICAL PSYCHOMETRICS (R5 vs R1/R2) Chi-Squared Test ---
  Metric      Chi2  P_Value Dominant_Category_in_R5  R5_Category_Pct Significant
0   disc  556.3072      0.0                      CI          10.6098           ✅
1   mbti  386.8177      0.0                    ENFP           8.5080           ✅


In [ ]:
# Recreate the df_analysis structure, including all categorical metrics for encoding
df_master = df_master_psych_analysis.copy()

# A. Create the PAPI pivoted structure
df_papi_pivoted = df_master.groupby(['employee_id', 'scale_code'])['score'].mean().unstack()

# B. Prepare Psych and Rating data (including categoricals for encoding)
psych_cols_for_merge = ['employee_id', 'rating', 'pauli', 'iq', 'gtq', 'tiki', 'disc', 'mbti']
df_psych_with_rating = df_master[psych_cols_for_merge].drop_duplicates(subset=['employee_id', 'rating'])

# C. Final Merge to create the base analysis table
df_analysis_base = df_papi_pivoted.merge(
    df_psych_with_rating,
    on='employee_id',
    how='inner'
)

# --- MODEL SETUP ---

# 1. Define Target Variable (1 = High Performer, 0 = Non-High Performer)
df_analysis_base['is_high_performer'] = np.where(df_analysis_base['rating'] == 5.0, 1, 0)
TARGET = 'is_high_performer'

# 2. Define Features
NUMERIC_FEATURES = [col for col in df_papi_pivoted.columns.tolist() if col in df_analysis_base.columns] + ['pauli', 'iq', 'gtq', 'tiki']
CATEGORICAL_FEATURES = ['disc', 'mbti'] # Include the significant categorical features

# 3. Handle Missing Values
# Drop rows with any missing values in the features used for the model
FEATURES_TO_CHECK = NUMERIC_FEATURES + CATEGORICAL_FEATURES + [TARGET]
df_model_ready = df_analysis_base[FEATURES_TO_CHECK].dropna()


# 4. One-Hot Encode Categorical Features (CRUCIAL STEP)
# Convert DISC and MBTI into binary (0 or 1) columns for the model
df_encoded = pd.get_dummies(df_model_ready, columns=CATEGORICAL_FEATURES, drop_first=True)


# 5. Define X and Y for the model
# X = All encoded numeric and categorical features
X = df_encoded.drop(columns=[TARGET])
# Y = The binary target
y = df_encoded[TARGET]


# 6. Standardize Numeric Features (CRUCIAL for comparing coefficients)
# Standardize only the numeric columns to prevent features with larger scales (like IQ) from dominating
scaler = StandardScaler()
X[NUMERIC_FEATURES] = scaler.fit_transform(X[NUMERIC_FEATURES])


# 7. Run Logistic Regression
# Use L2 penalty (Ridge) for robust coefficient estimation
model = LogisticRegression(penalty='l2', solver='liblinear', random_state=42)
model.fit(X, y)

# 8. Extract and Sort Coefficients
coefficients = pd.DataFrame({
    'Metric': X.columns,
    'Coefficient (Scaled)': model.coef_[0]
})

# Sort by absolute magnitude of the coefficient to find the strongest drivers
coefficients['Abs_Coeff'] = coefficients['Coefficient (Scaled)'].abs()
df_model_drivers = coefficients.sort_values('Abs_Coeff', ascending=False).drop(columns=['Abs_Coeff']).head(15)


print("\n--- TOP 15 PREDICTIVE DRIVERS OF RATING 5 (Logistic Regression) ---")
print("Interpretation: Positive Coeff = Trait increases the ODDS of achieving Rating 5.")
print("Interpretation: Negative Coeff = Trait decreases the ODDS of achieving Rating 5.")
print(df_model_drivers.round(4).to_string())


--- TOP 15 PREDICTIVE DRIVERS OF RATING 5 (Logistic Regression) ---
Interpretation: Positive Coeff = Trait increases the ODDS of achieving Rating 5.
Interpretation: Negative Coeff = Trait decreases the ODDS of achieving Rating 5.
       Metric  Coefficient (Scaled)
36  mbti_ENTJ               -0.9736
49  mbti_ISTP                0.7301
40  mbti_ESTJ               -0.6278
24    disc_CI                0.5343
38  mbti_ESFJ                0.5168
25    disc_CS               -0.4986
28    disc_DS                0.4528
31    disc_IS               -0.4114
46  mbti_ISFJ                0.3236
47  mbti_ISFP                0.2666
35  mbti_ENFP                0.2622
44  mbti_INTJ                0.2102
26    disc_DC                0.2044
32    disc_SC               -0.1968
17     Papi_W               -0.1860


In [12]:
df_drivers = df_model_drivers.copy()

# Create a color category based on the sign of the coefficient
df_drivers['Direction'] = df_drivers['Coefficient (Scaled)'].apply(lambda x: 'Positive Driver (R5 Increase)' if x > 0 else 'Negative Suppressor (R5 Decrease)')

# Sort by coefficient magnitude for the chart presentation
df_drivers = df_drivers.sort_values(by='Coefficient (Scaled)', ascending=True)

# Using the prepared df_drivers DataFrame
base = alt.Chart(df_drivers).encode(
    # Y-axis lists the metrics, sorted by the coefficient value
    y=alt.Y('Metric', sort=alt.EncodingSortField(field='Coefficient (Scaled)', op='mean', order='ascending'), title="PAPI/Psych Metric"),
    # Tooltip provides detail on hover
    tooltip=['Metric', alt.Tooltip('Coefficient (Scaled)', format=".4f"), 'Direction']
)

# 1. Bar Marks (The Coefficients)
bars = base.mark_bar().encode(
    x=alt.X('Coefficient (Scaled)', title="Standardized Predictive Strength (Coefficient)"),
    # Color based on the direction (Positive/Negative)
    color=alt.Color('Direction', 
                    scale=alt.Scale(domain=['Positive Driver (R5 Increase)', 'Negative Suppressor (R5 Decrease)'], 
                                    range=['#1f77b4', '#d62728']), 
                    title="Impact on Odds of Rating 5")
)

# 2. Zero Line (Reference)
zero_line = alt.Chart(pd.DataFrame({'zero': [0]})).mark_rule(color='black', strokeDash=[3, 3]).encode(
    x='zero:Q'
)

# Combine and finalize the chart
chart_altair = (zero_line + bars).properties(
    title="Top Drivers and Suppressors of High Performance (Rating 5)"
).interactive() # Allows interactive zooming/panning

chart_altair # Display the chart in the Jupyter Notebook

alt.LayerChart(...)

**Insights and Key Findings**

> We compared the cognitive ability and personality profiles of our Highest Performers (Rating 5) against our Lowest Performers (Rating 1 or 2).

> Finding: The difference between our best and worst employees is not based on general friendliness or basic work ethic; it’s that the top performers are significantly better endowed with raw intellectual capacity and possess specific analytical personality traits.

> Top 3 Traits to Prioritize for Performance

1. General Talent Quotient (GTQ) / Cognitive Power (The Gatekeeper). Top performers possess significantly higher GTQ scores, meaning they have superior analytical and complex problem-solving abilities. Actionable Insight: Cognitive ability is the essential entry ticket. When hiring or promoting for critical roles, GTQ should be the most important, non-negotiable metric. No amount of personality fit can fully compensate for a lack of intellectual horsepower.

2. MBTI: ISTP (The Analytical Executioner).This reserved, practical, and analytical profile is the single strongest positive personality driver of R5 success. Top performers are often "mechanics", focused on mastery, troubleshooting, and executing technical solutions with precision. Actionable Insight: When building high-stakes project teams, prioritize individuals who can demonstrate deep technical mastery and a calm, analytical approach to problem-solving. Success is driven by reliable specialists, not just general managers.

3. DISC: CI (Conscientiousness/Influence) (The Methodical Achiever).This profile combines a powerful focus on accuracy and high standards with an ability to communicate effectively. They ensure rigorous quality while still engaging stakeholders. Actionable Insight: This trait indicates that high performance demands a combination of methodical rigor and diplomatic engagement. Coach employees to uphold strict quality standards while effectively influencing peers and clients.

> Conclusion: The recipe for a Rating 5 is clear: You need High Cognitive Power GTQ paired with the Personality of a Master Executor ISTP or CI. Many common personality traits (like agreeableness or extraversion) do not independently influence top performance. Furthermore, traditionally dominant profiles MBTI: ENTJ and ESTJ are surprisingly less likely to achieve R5, suggesting the key is disciplined execution, not aggressive management.


## Behavioral Strengths Analysis

In [13]:
# --- CONFIGURATION ---
N_STRENGTHS = 5    # Number of top strengths to include
N_COMPETENCIES = 2 # Number of top competencies to include
TOP_MBTI = ['ISTP']  # Manually selected strong predictors
TOP_DISC = ['CI']    # Manually selected strong predictors
NUMERIC_PREDICTORS = ['gtq']

# Helper function to clean column names
def sanitize_column_name(name):
    return name.replace(' ', '_').replace('&', 'and')

# --- 1. Master Data Integration and Target Creation ---

# Combine Performance and PAPI/Competency Data to get base ratings and scores
df_master_comp = df_papi.merge(
    df_performance[['employee_id', 'rating', 'year']],
    on='employee_id',
    how='left'
)

# Use the latest rating for performance status
df_master_rating = df_master_comp.sort_values(['employee_id','year'], ascending=[True,False]).drop_duplicates('employee_id')[['employee_id', 'rating']]

# Merge with Psychometric Data (IQ, MBTI, DISC, GTQ)
df_master = df_master_rating.merge(
    df_psych[['employee_id', 'gtq', 'mbti', 'disc']].drop_duplicates(),
    on='employee_id',
    how='inner' # Only include employees with both rating and psych data
)

# Create the Binary Target Variable
TARGET_VARIABLE = 'is_R5'
df_master[TARGET_VARIABLE] = np.where(df_master['rating'] == 5.0, 1, 0)
df_master = df_master.drop_duplicates(subset=['employee_id']).reset_index(drop=True)

# --- 2. DYNAMIC PREDICTOR SELECTION ---

# 2A. Dynamic Selection of TOP_STRENGTHS (Based on Odds Ratio)
print("1. Dynamically selecting top N strengths (based on OR)...")
df_perf_latest = df_performance.sort_values(['employee_id','year'], ascending=[True,False]).drop_duplicates('employee_id')[['employee_id', 'rating']]
df_strengths_with_rating = df_strengths[df_strengths['rank'] < 3].merge(
    df_perf_latest, on='employee_id', how='left'
)
df_strengths_with_rating['is_R5'] = np.where(df_strengths_with_rating['rating'] == 5.0, 1, 0)
df_strengths_with_rating['is_low_performer'] = np.where(df_strengths_with_rating['rating'].isin([1.0, 2.0]), 1, 0)

df_analysis = df_strengths_with_rating[
    (df_strengths_with_rating['is_R5'] == 1) | (df_strengths_with_rating['is_low_performer'] == 1)
].copy()

employee_master_strengths = df_analysis[['employee_id', 'is_R5']].drop_duplicates()
TOTAL_R5 = employee_master_strengths['is_R5'].sum()
TOTAL_NONR5 = len(employee_master_strengths) - TOTAL_R5

df_theme_presence = df_analysis.groupby(['employee_id', 'theme']).size().unstack(fill_value=0)
df_theme_presence = (df_theme_presence > 0).astype(int) 
df_theme_presence = employee_master_strengths.merge(
    df_theme_presence, on='employee_id', how='inner'
).drop(columns=['employee_id'])

odds_ratio_results = []
theme_list = [col for col in df_theme_presence.columns if col != 'is_R5']
for theme in theme_list:
    a = df_theme_presence[(df_theme_presence['is_R5'] == 1) & (df_theme_presence[theme] == 1)].shape[0]
    b = df_theme_presence[(df_theme_presence['is_R5'] == 0) & (df_theme_presence[theme] == 1)].shape[0]
    c = TOTAL_R5 - a
    d = TOTAL_NONR5 - b
    if a == 0 or b == 0 or c == 0 or d == 0:
        a += 0.5; b += 0.5; c += 0.5; d += 0.5
    odds_ratio = (a * d) / (b * c)
    odds_ratio_results.append({'Theme': theme, 'Odds_Ratio': odds_ratio})

df_odds_ratio = pd.DataFrame(odds_ratio_results)
TOP_STRENGTHS = df_odds_ratio.sort_values(by='Odds_Ratio', ascending=False).head(N_STRENGTHS)['Theme'].tolist()
print(f"   Selected TOP_STRENGTHS ({N_STRENGTHS}): {TOP_STRENGTHS}")


# 2B. Dynamic Selection of TOP_COMPETENCIES (Based on Cohen's d)
print("2. Dynamically selecting top N competencies (based on Cohen's d)...")
df_all_scores = df_master_comp[['employee_id', 'scale_code', 'score']].merge(
    df_master[['employee_id', 'is_R5']].drop_duplicates(),
    on='employee_id',
    how='inner'
)

competency_d_results = []
all_competencies = df_all_scores['scale_code'].unique()

for comp in all_competencies:
    scores_r5 = df_all_scores[(df_all_scores['scale_code'] == comp) & (df_all_scores['is_R5'] == 1)]['score'].dropna()
    scores_not_r5 = df_all_scores[(df_all_scores['scale_code'] == comp) & (df_all_scores['is_R5'] == 0)]['score'].dropna()
    
    if len(scores_r5) < 2 or len(scores_not_r5) < 2:
        continue

    mean_diff = scores_r5.mean() - scores_not_r5.mean()
    n1, n2 = len(scores_r5), len(scores_not_r5)
    std1, std2 = scores_r5.std(), scores_not_r5.std()
    
    pooled_std = np.sqrt(((n1 - 1) * std1**2 + (n2 - 1) * std2**2) / (n1 + n2 - 2))
    cohens_d = mean_diff / pooled_std if pooled_std > 0 else 0
    
    competency_d_results.append({'Competency': comp, 'Cohens_D': cohens_d})

df_competency_ranking = pd.DataFrame(competency_d_results)
TOP_COMPETENCIES = df_competency_ranking.iloc[
    df_competency_ranking['Cohens_D'].abs().argsort()[::-1]
].head(N_COMPETENCIES)['Competency'].tolist()
print(f"   Selected TOP_COMPETENCIES ({N_COMPETENCIES}): {TOP_COMPETENCIES}")


# --- 3. FINAL FEATURE PREPARATION (Pivoting and Merging) ---

# 3A. Competency Score Preparation (Pivoting)
df_competency_pivot = df_master_comp.pivot_table(
    index='employee_id',
    columns='scale_code',
    values='score',
    aggfunc='mean'
).reset_index()

comp_name_map = {
    col_orig: sanitize_column_name(col_orig) 
    for col_orig in df_competency_pivot.columns 
    if col_orig in TOP_COMPETENCIES
}

df_competency_scores = df_competency_pivot[['employee_id'] + list(comp_name_map.keys())].rename(columns=comp_name_map)
df_master = df_master.merge(df_competency_scores, on='employee_id', how='left')

# 3B. Strengths Dummy Preparation
df_strength_pivot = df_strengths[df_strengths['rank'] < 3].assign(value=1).pivot_table(
    index='employee_id', 
    columns='theme', 
    values='value', 
    aggfunc='max'
).fillna(0).reset_index()

df_strength_dummies = df_strength_pivot[['employee_id'] + TOP_STRENGTHS]
df_master = df_master.merge(df_strength_dummies, on='employee_id', how='left').fillna(0) 

# 3C. Categorical Dummies
for mbti_type in TOP_MBTI:
    df_master[f'mbti_{mbti_type}'] = np.where(df_master['mbti'] == mbti_type, 1, 0)

for disc_type in TOP_DISC:
    df_master[f'disc_{disc_type}'] = np.where(df_master['disc'] == disc_type, 1, 0)


# --- 4. INTEGRATED LOGISTIC REGRESSION ---

# Compile Final Predictor List
COMPETENCY_PREDICTORS = [sanitize_column_name(c) for c in TOP_COMPETENCIES]
STRENGTH_PREDICTORS = TOP_STRENGTHS
MBTI_DISC_PREDICTORS = [f'mbti_{mbti}' for mbti in TOP_MBTI] + [f'disc_{disc}' for disc in TOP_DISC]

ALL_PREDICTORS = NUMERIC_PREDICTORS + COMPETENCY_PREDICTORS + STRENGTH_PREDICTORS + MBTI_DISC_PREDICTORS

# Prepare Data (Dropping NaNs is critical for regression)
df_final_model = df_master[[TARGET_VARIABLE] + ALL_PREDICTORS].dropna()

Y = df_final_model[TARGET_VARIABLE]
X = df_final_model[ALL_PREDICTORS]
X = sm.add_constant(X, prepend=False)

# Run the Model
print("\n4. Fitting Final Integrated Logistic Regression...")
model = sm.Logit(Y, X)
result = model.fit(disp=False)
print("Fitting Complete.")

# --- 5. EXTRACT AND VISUALIZE RESULTS ---

# Calculate Odds Ratios and P-Values
results_df = pd.DataFrame({
    'Metric': result.params.index,
    'Odds_Ratio': np.exp(result.params.values),
    'P_Value': result.pvalues.values
})

results_df = results_df[results_df['Metric'] != 'const']

# Determine Significance and Color for Visualization
results_df['Significant'] = np.where(results_df['P_Value'] < 0.05, '✅', '❌')
results_df['Significance_Color'] = np.where(results_df['P_Value'] < 0.05, 'Significant (P < 0.05)', 'Not Significant (P ≥ 0.05)')

df_final_ranking = results_df.sort_values(by='Odds_Ratio', ascending=False).round(4)

print("\n--- 🏆 Final Drivers of Rating 5 (Integrated Model) ---")
print("These are the unique, independent predictors of R5 performance.")
print(df_final_ranking.to_string(index=False))

1. Dynamically selecting top N strengths (based on OR)...
   Selected TOP_STRENGTHS (5): ['Futuristic', 'Learner', 'Relator', 'Intellection', 'Activator']
2. Dynamically selecting top N competencies (based on Cohen's d)...
   Selected TOP_COMPETENCIES (2): ['Papi_P', 'Papi_S']

4. Fitting Final Integrated Logistic Regression...
Fitting Complete.

--- 🏆 Final Drivers of Rating 5 (Integrated Model) ---
These are the unique, independent predictors of R5 performance.
      Metric  Odds_Ratio  P_Value Significant         Significance_Color
  Futuristic      2.2358   0.0050           ✅     Significant (P < 0.05)
     Learner      1.8201   0.0405           ✅     Significant (P < 0.05)
   Activator      1.6444   0.1226           ❌ Not Significant (P ≥ 0.05)
     Relator      1.6335   0.1139           ❌ Not Significant (P ≥ 0.05)
   mbti_ISTP      1.4548   0.2250           ❌ Not Significant (P ≥ 0.05)
Intellection      1.3348   0.3640           ❌ Not Significant (P ≥ 0.05)
      Papi_P      1.0

In [14]:
# --- Visualization (Odds Ratio Ranking) ---

# Set a baseline for the Y-axis (OR = 1.0)
base = alt.Chart(df_final_ranking).encode(
    y=alt.Y('Metric', sort=alt.EncodingSortField(field='Odds_Ratio', op='mean', order='descending'), title='Predictor Metric'),
    tooltip=['Metric', alt.Tooltip('Odds_Ratio', format='.3f'), alt.Tooltip('P_Value', format='.4f'), 'Significant']
).properties(
    title='Integrated Model: Predictive Odds Ratio for Rating 5'
).interactive()

# Bar Chart for Magnitude (Odds Ratio)
bars = base.mark_bar().encode(
    x=alt.X('Odds_Ratio', title='Odds Ratio (e^Coefficient)'),
    color=alt.Color('Significance_Color', 
                    scale=alt.Scale(domain=['Significant (P < 0.05)', 'Not Significant (P ≥ 0.05)'], 
                                    range=['#2ca02c', '#ff7f0e']), 
                    title='Statistical Significance')
)

# Reference Line at OR = 1.0
reference_line = alt.Chart(pd.DataFrame({'OR_1': [1]})).mark_rule(color='black', strokeDash=[3, 3]).encode(
    x='OR_1:Q'
)

# Combine the bars and the reference line
chart_odds_ratio = (reference_line + bars).properties(width=600)

chart_odds_ratio

alt.LayerChart(...)

**Insights and Key Findings**

> Finding: The difference between top performers and others is overwhelmingly defined by a future-oriented, growth mindset, not just raw intelligence. The effect of GTQ and most personality profiles completely disappears once behavioral strengths are accounted for.

> Top 2 Non-Redundant Behavioral Drivers (The Core of R5)
1. Futuristic (The Visionary Accelerator). This is the single strongest independent predictor in the entire model. It proves top performance is driven by a mindset focused on long-term strategy and conceiving future possibilities. Actionable Insight: Prioritize employees who naturally look ahead, conceptualize future possibilities, and drive long-term strategic value. This theme must be a core focus for hiring and promotion into innovation and leadership roles.

2. Learner (The Continuous Improver). This relentless drive for skill mastery and knowledge acquisition is the second core ingredient for sustained top performance. Actionable Insight: We must recruit and coach individuals who see their work as an ongoing challenge to learn and improve. Incentivize skill acquisition to ensure employees remain relevant and valuable over time.

> Conclusion: What Matters: The most successful employees are proactive, future-focused individuals who prioritize continuous learning Futuristic and Learner. What Doesn't Matter: The effect of raw intelligence GTQ and most personality types disappears when these behavioral themes are present.

## Contextual Factors Analysis

In [15]:
# df_master from merge Structural Context tables (Grade, Position, Department, etc.)
df_master = df_performance.merge(df_employees, on='employee_id', how='left')
df_master = df_master.merge(df_grades, on='grade_id', how='left', suffixes=('', '_grade'))
df_master = df_master.merge(df_positions, on='position_id', how='left', suffixes=('', '_position'))
df_master = df_master.merge(df_departments, on='department_id', how='left', suffixes=('', '_dept'))
df_master = df_master.merge(df_education, on='education_id', how='left', suffixes=('', '_education'))

# Data Cleaning: Ensure 'rating' is numeric and valid
df_master['rating'] = pd.to_numeric(df_master['rating'], errors='coerce')

# Filter hanya rating yang valid (1.0 hingga 5.0)
VALID_RATINGS = [1.0, 2.0, 3.0, 4.0, 5.0]

if 'name' in df_master.columns:
    df_master = df_master.rename(columns={'name': 'grade_name'})

# Membuat DataFrame sementara yang sudah bersih
df_clean = df_master[df_master['rating'].isin(VALID_RATINGS)].copy()

In [ ]:
# --- CONFIGURATION: Define Specific Predictors (Based on your input) ---
NUMERIC_CONTEXT_PREDICTORS = ['years_of_service_months']
CATEGORICAL_CONTEXT_PREDICTORS = ['grade_name', 'name_education'] 

# --- 1. DATA FIX: Filter to ONE ROW PER EMPLOYEE (LATEST RATING) ---

# We use df_clean, which contains historical data, and filter it down to the latest record per employee.
df_context = (
    df_clean
    .sort_values(['employee_id', 'year'], ascending=[True, False])
    .drop_duplicates('employee_id', keep='first')
    .copy()
)

# 1B. Create the Binary Target Variable
TARGET_VARIABLE = 'is_R5'
df_context[TARGET_VARIABLE] = np.where(df_context['rating'] == 5.0, 1, 0)

# 1C. Prepare Categorical Factors for Model
df_dummies = pd.get_dummies(
    df_context[CATEGORICAL_CONTEXT_PREDICTORS], 
    drop_first=True, 
    prefix=CATEGORICAL_CONTEXT_PREDICTORS
)

# Merge dummies back into the context DataFrame
df_model_context = pd.concat([df_context, df_dummies], axis=1)

# Compile the final list of dummy columns
DUMMY_COLUMNS = list(df_dummies.columns)
FINAL_PREDICTORS = NUMERIC_CONTEXT_PREDICTORS + DUMMY_COLUMNS

print(f"Analysis running on {len(df_context)} unique employees (using latest rating).")

# --- 2. Logistic Regression (With Value Error Fix) ---

print("Filtering data and preparing for model...")
# Drop rows with any missing data across the target and all required predictors
df_final_model = df_model_context[[TARGET_VARIABLE] + FINAL_PREDICTORS].dropna()

# --- VALUE ERROR FIX: Ensure all predictors and target are numeric ---
for col in NUMERIC_CONTEXT_PREDICTORS:
    # Coerce non-numeric values in the numeric column to NaN (which are already dropped)
    df_final_model[col] = pd.to_numeric(df_final_model[col], errors='coerce') 

Y = df_final_model[TARGET_VARIABLE].astype(int) # Target must be integer
X = df_final_model[FINAL_PREDICTORS].astype(float) # Predictors must be float

# Add a constant term (intercept)
X = sm.add_constant(X, prepend=False)

# Run the Model
print("Fitting Contextual Factors Logistic Regression Model...")
model = sm.Logit(Y, X)
result = model.fit(disp=False)
print("Fitting Complete.")

# --- 3. Extract and Visualize Results ---

# Calculate Odds Ratios and P-Values
results_df = pd.DataFrame({
    'Metric': result.params.index,
    'Odds_Ratio': np.exp(result.params.values),
    'P_Value': result.pvalues.values
})

# Filter out the constant term and determine significance
results_df = results_df[results_df['Metric'] != 'const']
results_df['Significant'] = np.where(results_df['P_Value'] < 0.05, '✅', '❌')
results_df['Significance_Color'] = np.where(results_df['P_Value'] < 0.05, 'Significant (P < 0.05)', 'Not Significant (P ≥ 0.05)')

df_final_ranking = results_df.sort_values(by='Odds_Ratio', ascending=False).round(4)

print("\n--- 🏆 Predictive Contextual Factors Ranking (Odds Ratio) ---")
print("Interpretation: OR > 1.0 increases odds of R5, OR < 1.0 decreases odds.")
print(df_final_ranking.to_string(index=False))


# --- 4. Visualization (Odds Ratio Forest Plot) ---

# Set a baseline for the Y-axis (OR = 1.0)
base = alt.Chart(df_final_ranking).encode(
    y=alt.Y('Metric', sort=alt.EncodingSortField(field='Odds_Ratio', op='mean', order='descending'), title='Contextual Factor'),
    tooltip=['Metric', alt.Tooltip('Odds_Ratio', format='.3f'), alt.Tooltip('P_Value', format='.4f'), 'Significant']
).properties(
    title='Contextual Factors: Odds Ratio for Rating 5'
).interactive()

# Bar Chart for Magnitude (Odds Ratio)
bars = base.mark_bar().encode(
    x=alt.X('Odds_Ratio', title='Odds Ratio (e^Coefficient)'),
    color=alt.Color('Significance_Color', 
                    scale=alt.Scale(domain=['Significant (P < 0.05)', 'Not Significant (P ≥ 0.05)'], 
                                    range=['#2ca02c', '#ff7f0e']), # Green/Orange color scheme
                    title='Statistical Significance')
)

# Reference Line at OR = 1.0
reference_line = alt.Chart(pd.DataFrame({'OR_1': [1]})).mark_rule(color='black', strokeDash=[3, 3]).encode(
    x='OR_1:Q'
)

# Combine the bars and the reference line
chart_odds_ratio = (reference_line + bars).properties(width=600)

chart_odds_ratio

Analysis running on 2010 unique employees (using latest rating).
Filtering data and preparing for model...
Fitting Contextual Factors Logistic Regression Model...
Fitting Complete.

--- 🏆 Predictive Contextual Factors Ranking (Odds Ratio) ---
Interpretation: OR > 1.0 increases odds of R5, OR < 1.0 decreases odds.
                 Metric  Odds_Ratio  P_Value Significant         Significance_Color
      name_education_S2      1.9730   0.0033           ✅     Significant (P < 0.05)
     name_education_SMA      1.9326   0.0056           ✅     Significant (P < 0.05)
      name_education_S1      1.5882   0.0555           ❌ Not Significant (P ≥ 0.05)
          grade_name_IV      1.3079   0.1522           ❌ Not Significant (P ≥ 0.05)
           grade_name_V      1.0654   0.7455           ❌ Not Significant (P ≥ 0.05)
years_of_service_months      1.0029   0.3387           ❌ Not Significant (P ≥ 0.05)


alt.LayerChart(...)

**Insight and Key Findings**

> Finding: The only factors that are statistically reliable drivers of top performance are specific levels of Educational Attainment. Grade and Tenure do not independently predict who achieves a Rating 5.

> Top Drivers of Rating 5
1. Education S2 (The Master's Advantage). Having a Master's degree gives an employee a significantly higher chance of achieving a Rating 5 compared to the baseline education level. This is the strongest structural predictor you have.Action: Prioritize the hiring and attraction of talent with postgraduate degrees. These candidates represent the highest probability of achieving top performance.

2. Education SMA (The High School Excellence). Having a High School degree (SMA) also gives an employee a much higher chance of achieving a Rating 5 compared to the baseline. This reveals a surprisingly high-potential talent pool. Action: Investigate this high-performing SMA group further (e.g., check their natural ability or strengths). They may offer an excellent talent source that excels due to strong inherent talent despite having less formal schooling.

> Conclusion: Education is the dominant structural factor. We must stop relying on traditional metrics like tenure and internal grade movement to predict success, as the data proves they are non-factors. Instead, the strategy must prioritize the proven high-potential groups defined by the S2 and SMA education levels.

# The Rating 5 Success Formula: Rule-Based Logic

## **The Most Critical Findings**

The following points synthesize the most powerful and reliable insights from our **Competency, Psychometric, Behavioral, and Contextual Factor** analyses, demonstrating how performance emerges from multiple factors.

> 1. The Core Behavioral Mindset is Non-Negotiable (The Engine)

This finding comes from the **Behavioral Strengths Analysis**, which tested all factors simultaneously.

* **The Finding:** When intelligence ($\text{GTQ}$), competencies, and all other factors are controlled for, the **Futuristic** and **Learner** themes are the **only factors** that remain statistically significant and independently accelerate performance.
* **The Takeaway:** **Mindset is the primary driver.** No amount of competence or experience can compensate for a lack of a future-focused, growth-oriented mindset. This must be the foundation of your talent strategy.
* **Formula Justification:** This is why the **Behavioral Mindset** receives the majority weight ($\mathbf{75\%}$ in the Success Formula)—it is the **true, non-redundant engine** of $\text{R5}$ performance.

---

> 2. Execution Competency Defines the Output (The Transmission)

This finding comes from the **Competency Pillars Analysis** ($\text{R5}$ vs. $\text{R1/R2}$).

* **The Finding:** The largest performance gaps were found in the $\text{Value Creation for Users}$ and $\text{Insight \& Decision Sharpness}$ competencies.
* **The Takeaway:** $\text{R5}$ is defined by **impact, not just effort**. The top performers excel at translating their mindset into measurable value and clear, correct decisions.
* **Formula Justification:** The $\mathbf{15\%}$ weight assigned to **Execution Competency** ensures the formula requires employees to have the necessary skills to deliver the superior outcomes demanded by their $\text{R5}$ Mindset.

---

> 3. The Structural Filters Are Surprising and Narrow (The Chassis)

This finding comes from the **Contextual Factor Analysis** (testing Grade, Tenure, Education).

* **The Finding:** The only factors that act as significant structural filters are specific **Education Levels ($\text{S2}$ and $\text{SMA}$)**. Factors like **Years of Service (Tenure) and Grade Level are proven to be irrelevant** as independent predictors.
* **The Takeaway:** **Tenure and Grade are misleading metrics.** You should immediately stop relying on them for talent prediction. Strategy should focus on two distinct, high-potential talent pools defined by their education.
* **Formula Justification:** This insight allows us to **eliminate non-factors** and assign the smallest, but still necessary, weight ($\mathbf{10\%}$) to the few structural requirements that actually matter.

## **The Success Formula Structure**
The formula establishes the relative importance of talent factors based on their statistical power to drive Rating 5. 

$$\mathbf{\text{Rating 5 Potential}} = \mathbf{75\% \text{ Behavioral Mindset}} + \mathbf{15\% \text{ Execution Competency}} + \mathbf{10\% \text{ Structural Filter}}$$


**Justification for Weights (Why These Percentages?)**

The weights assigned are a management decision directly reflecting the statistical hierarchy proven by the data:
* 75% Behavioral Mindset: This received the highest weight because the Futuristic and Learner themes were the only statistically significant, non-redundant accelerators in the final, integrated analysis. They are the core engine of sustained $\text{R5}$ performance.
* 15% Execution Competency: Represents essential execution skills (e.g., $\text{Value Creation}$) which showed the largest performance gaps in the initial competency analysis. These skills are necessary to translate the mindset into tangible results.
* 10% Structural Filter: Represents the foundational requirement of the $\text{S2}$ and $\text{SMA}$ education levels, which were the only significant structural predictors.